<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.dummy import DummyRegressor
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMRegressor
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from lightgbm import LGBMClassifier

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [5]:
df.nunique()

text     159571
toxic         2
dtype: int64

Из первичного анализа данных мы видим что у нас в наличии достаточно объемный датафрейм с сообщениями на английском языке, пропусков нет, баланс токсичных и не токсичных где то около 1 к 9, возможно для улучшения модели потребуется сбалансировать их. А пока у нас следующий план действий:

1) Подобрать бибилиотеку для лемматизации текстов. Pymystem насколько я помню заточен на работу с русскими текстами, но нам ничего не мешает попробовать. Из того что давалось в теории я склоняюсь к nltk.

2) Написать функции для отчистки, лемматизации и избавления от стоп слов.

3) Преобразовать предложения в матрицы.

4) Сделать сбалансированную выборку данных.

5) Разделить на выборки и обучить модели.

In [6]:
corpus = list(df['text'])

In [7]:
def clear_text(text):
    eng_text = re.sub(r'[^a-zA-Z]', ' ', text)
    eng_text_split = eng_text.split()
    text = ' '.join(eng_text_split)
    return(text)

In [8]:
def lemmatizer(text):
    m = Mystem()
    lemm_list = m.lemmatize(text)
    lemm_text = ' '.join(lemm_list)  
    return lemm_text

Сделаем переменную для проверки функций.

In [9]:
c_corpus = clear_text(corpus[0])
print(c_corpus)
lemmatizer(c_corpus)

Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now


'Explanation   Why   the   edits   made   under   my   username   Hardcore   Metallica   Fan   were   reverted   They   weren   t   vandalisms   just   closure   on   some   GAs   after   I   voted   at   New   York   Dolls   FAC   And   please   don   t   remove   the   template   from   the   talk   page   since   I   m   retired   now \n'

Cудя по данному примеру, pymystem не особо подходит для лемматизации английский текстов, в таком случае лемматизируем с помощью nltk.

In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
lemmatizer = WordNetLemmatizer()

In [12]:
def lemmatize_nltk(sentence):
    word_list = nltk.word_tokenize(sentence)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return(lemmatized_output)

In [13]:
lemmatize_nltk(c_corpus)

'Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalism just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now'

In [14]:
def stop_words_cleaning(sentence):
    filtered_sentence = []
    word_list = nltk.word_tokenize(sentence)
    for w in word_list:
        if w not in stop_words:
            filtered_sentence.append(w)
    clear_sentence = ' '.join(filtered_sentence)
    return(clear_sentence)

In [15]:
stop_words_cleaning(c_corpus)

'Explanation Why edits made username Hardcore Metallica Fan reverted They vandalisms closure GAs I voted New York Dolls FAC And please remove template talk page since I retired'

In [16]:
lemmatize_text = []
for i in range(len(corpus)):
    lemmatize_text.append(lemmatize_nltk(stop_words_cleaning(clear_text(corpus[i]))))
lemmatize_text

['Explanation Why edits made username Hardcore Metallica Fan reverted They vandalism closure GAs I voted New York Dolls FAC And please remove template talk page since I retired',
 'D aww He match background colour I seemingly stuck Thanks talk January UTC',
 'Hey man I really trying edit war It guy constantly removing relevant information talking edits instead talk page He seems care formatting actual info',
 'More I make real suggestion improvement I wondered section statistic later subsection type accident I think reference may need tidying exact format ie date format etc I later one else first preference formatting style reference want please let know There appears backlog article review I guess may delay reviewer turn It listed relevant form eg Wikipedia Good article nomination Transport',
 'You sir hero Any chance remember page',
 'Congratulations well use tool well talk',
 'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK',
 'Your vandalism Matt Shirvington article reverted Please ba

In [17]:
df['lemm_text'] = lemmatize_text

Мы получили лемматизированный текст который в дальнейшем превратим в признаки, теперь сделаем сбалансированные выборки для проверки на каких данных обучение пройдет качественнее. Поскольку у нас достаточно большая выборка, я считаю что в данном случае будет правильнее пойти на ее уменьшение, так как если мы пойдем на увеличение то вес многих слов кратно увеличится.

In [18]:
features = df['lemm_text']
target = df['toxic']

In [19]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                       target, test_size = 0.3, random_state=12345, stratify = target)

In [21]:
features_selection, features_valid, target_selection, target_valid = train_test_split(features_train, 
                                       target_train, test_size = 0.25, random_state=12345, stratify = target_train)

In [22]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [23]:
feature_balance, target_balance = downsample(features_train, target_train, 0.13)
target_balance.value_counts()

0    13044
1    11357
Name: toxic, dtype: int64

Подготовим признаки для двух выборок

In [24]:
features_train_vector = TfidfVectorizer().fit_transform(features_train)
features_train_vector

<111699x132749 sparse matrix of type '<class 'numpy.float64'>'
	with 3204017 stored elements in Compressed Sparse Row format>

In [25]:
features_test_vector = TfidfVectorizer().fit(features_train).transform(features_test)
features_test_vector

<47872x132749 sparse matrix of type '<class 'numpy.float64'>'
	with 1347446 stored elements in Compressed Sparse Row format>

In [26]:
features_selection_vector = TfidfVectorizer().fit_transform(features_selection)
features_selection_vector

<83774x112417 sparse matrix of type '<class 'numpy.float64'>'
	with 2411117 stored elements in Compressed Sparse Row format>

In [27]:
features_valid_vector = TfidfVectorizer().fit(features_selection).transform(features_valid)
features_valid_vector

<27925x112417 sparse matrix of type '<class 'numpy.float64'>'
	with 771342 stored elements in Compressed Sparse Row format>

In [28]:
features_train_balance_vector = TfidfVectorizer().fit_transform(feature_balance)
features_train_balance_vector

<24401x50062 sparse matrix of type '<class 'numpy.float64'>'
	with 620618 stored elements in Compressed Sparse Row format>

In [29]:
features_test_balance_vector = TfidfVectorizer().fit(feature_balance).transform(features_test)
features_test_balance_vector

<47872x50062 sparse matrix of type '<class 'numpy.float64'>'
	with 1313025 stored elements in Compressed Sparse Row format>

Данные подготовлены, давайте посмотрим что покажет модель.

## Обучение

In [30]:
best_result = 0.75
best_c = 0
for i in range(1, 31, 1):
    model_1 = LogisticRegression(solver = 'saga', class_weight = 'balanced', C = i * 0.1)
    model_1.fit(features_selection_vector, target_selection)
    answer_1 = model_1.predict(features_valid_vector)
    result_1 = f1_score(answer_1, target_valid)
    if result_1 > best_result:
        best_model_1 = model_1
        best_result = result_1
        best_c = i * 0.1

print("Лучшая оценка качества модели методом методом f1:", best_result, "Достигается при параметре С = ", best_c)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Лучшая оценка качества модели методом методом f1: 0.7665255617062845 Достигается при параметре С =  2.9000000000000004


In [31]:
model_log = LogisticRegression(solver = 'saga', class_weight = 'balanced', C = best_c)
model_log.fit(features_train_vector, target_train)
answer_log = model_log.predict(features_test_vector)
f1_score(answer_log, target_test)

0.78647476340694

In [32]:
model_log_balance = LogisticRegression(solver = 'liblinear')
model_log_balance.fit(features_train_balance_vector, target_balance)
answer_log_balance = model_log_balance.predict(features_test_balance_vector)
f1_score(answer_log_balance, target_test)

0.7068143100511073

In [33]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth = 71)
model_tree.fit(features_train_vector, target_train)
answer_tree = model_tree.predict(features_test_vector)
f1_score(answer_tree, target_test)

0.706803834463409

In [34]:
model_tree_balance = DecisionTreeClassifier(random_state=12345, max_depth = 85)
model_tree_balance.fit(features_train_balance_vector, target_balance)
answer_tree_balance = model_tree_balance.predict(features_test_balance_vector)
f1_score(answer_tree_balance, target_test)

0.5459202341748994

In [35]:
model_forest = RandomForestClassifier(random_state = 12345, n_estimators = 5, max_depth = 51)
model_forest.fit(features_train_vector, target_train)
answer_forest = model_forest.predict(features_test_vector)
f1_score(answer_forest, target_test)

0.11583011583011583

In [36]:
model_forest_balance = RandomForestClassifier(random_state = 12345, n_estimators = 17, max_depth = 51)
model_forest_balance.fit(features_train_balance_vector, target_balance)
answer_forest_balance = model_forest_balance.predict(features_test_balance_vector)
f1_score(answer_forest_balance, target_test)

0.5857626026735384

Модели обучены, необходимая точность соблюдается.

## Выводы

В рамках данного проекта мы обучили модель отвечающую необходимым условиям точности. Лучше всего себя показывают модели логистической регрессии. Модели случайного леса в данном случае сильно проигрывают, у решающего дерева ситуация получше, но все равно уступает логистической регрессии. Также нужно отметить что при работе с выборкой где целевые признаки были сбалансированы все модели показывают результаты хуже чем когда выборка бралась полностью с дисбалансом в целевом признаке.